# Search for products by tile

## Built-in search by tile

Many providers already support search by tile (*Sentinel 2 MGRS tiling grid*):
`peps`, `creodias`, `cop_dataspace`, `planetary_computer`, `earth_search`.

For these providers, you can use `grid:code` as search parameter using EODAG:

In [9]:
from eodag import EODataAccessGateway, setup_logging

dag = EODataAccessGateway()
products = dag.search(
    provider="peps",
    collection="S2_MSI_L1C", 
    start="2018-06-01", 
    end="2018-06-15", 
    **{"grid:code": "MGRS-31TFK"}
)
products

SearchResult([EOProduct(id=S2A_MSIL1C_20180614T103021_N0500_R108_T31TFK_20230812T193420, provider=peps),
              EOProduct(id=S2B_MSIL1C_20180612T104019_N0500_R008_T31TFK_20230824T092151, provider=peps),
              EOProduct(id=S2B_MSIL1C_20180609T103019_N0500_R108_T31TFK_20230823T173246, provider=peps),
              EOProduct(id=S2A_MSIL1C_20180607T104021_N0500_R008_T31TFK_20230818T193607, provider=peps),
              EOProduct(id=S2A_MSIL1C_20180604T103021_N0500_R108_T31TFK_20230819T115015, provider=peps),
              EOProduct(id=S2B_MSIL1C_20180602T104019_N0500_R008_T31TFK_20230827T165546, provider=peps)])

In [4]:
[p.properties["grid:code"] for p in products]

['MGRS-31TFK',
 'MGRS-31TFK',
 'MGRS-31TFK',
 'MGRS-31TFK',
 'MGRS-31TFK',
 'MGRS-31TFK']

## Search by tile using custom locations
`eodag` allows to search for products by geometric features that match a *location query*, see the [API user guide](../api_user_guide/3_search.ipynb#locations-search) for an introduction to this concept.

In this tutorial we will use a shapefile that represents the Sentinel 2 tiling grid to search for *Sentinel 2 Level-1C* products with *PEPS* **at a specific tile**. In this shapefile each tile is defined by its centroid and a `tile_id` attribute (e.g. *29PMT*). This shapefile was created by downloading first the Sentinel 2 tiling grid (MGRS) provided [by ESA as a KML file](https://web.archive.org/web/20200907072744/https://sentinel.esa.int/web/sentinel/missions/sentinel-2/data-products). It was then converted as a shapefile and processed to compute the centroids. We use the tile's centroid here as `eodag` returns products that intersects the user defined search area. Since tiles overlap with each other, using the polygons instead of the centroids would return more tiles than just the one we target. 

In [5]:
import os
from zipfile import ZipFile

# Interactive mapping
import folium
from folium.plugins import TimestampedGeoJson
# pyshp: to read shapefiles
import shapefile

### Setup

A workspace directory is created to store the files that will be generated.

In [6]:
workspace = "eodag_workspace_locations_tiles"
if not os.path.isdir(workspace):
    os.mkdir(workspace)

You should have an `auxdata` folder next to this tutorial's file. It contains a shapefile that is needed to run this tutorial correctly.

In [7]:
sentinel2_grid_zip = os.path.join("auxdata", "sentinel2_tiling_grid_centroids.zip")
if not os.path.isfile(sentinel2_grid_zip):
    raise FileNotFoundError("Auxdata not found, please check your configuration.")

In [8]:
# We unzip the archived shapefile.
with ZipFile(sentinel2_grid_zip, "r") as fzip:
    fzip.extractall("auxdata")

In this tutorial products will just be searched for, not downloaded. We don't need to set up PEPS credentials to search for products. If you wish to download them, you should set the credentials beforehand, using these two environment variables for instance.

In [6]:
# os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
# os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

Logging is activated to better inspect what `eodag` does internally.

In [10]:
setup_logging(2)  # INFO level

The default search criteria consists of a time period in June 2018 and `eodag`'s collection identifier for *Sentinel 2 Level-1C* products.

In [11]:
default_search_criteria = dict(
    collection="S2_MSI_L1C",
    start="2018-06-01",
    end="2018-06-15"
)

### Add a locations configuration

We check and store the content of this shapefile.

In [12]:
sentinel2_shp = os.path.join('auxdata', 'sentinel2_tiling_grid_centroids.shp')
with shapefile.Reader(sentinel2_shp) as shp:
    print(shp, "\n")
    print("fields:", shp.fields)
    shaperecs = shp.shapeRecords()

shapefile Reader
    56686 shapes (type 'POINT')
    56686 records (2 fields) 

fields: [('DeletionFlag', 'C', 1, 0), ['tile_id', 'C', 5, 0]]


It has about 57 000 tiles/polygons and a field `tile_id`.

We create a YAML file to configure this new location selector, we will refer to it with `s2_tile_centroid`.

In [13]:
# Save the locations configuration file.
locations_yaml_content = """
shapefiles:
  - name: s2_tile_centroid
    path: {}
    attr: tile_id
""".format(os.path.abspath(sentinel2_shp))

locations_filepath = os.path.abspath(os.path.join(workspace, "custom_locations.yml"))

with open(locations_filepath, "w") as f_yml:
    f_yml.write(locations_yaml_content.strip())

An instance of an [EODataAccessGateway](../../api_reference/core.rst#eodag.api.core.EODataAccessGateway) class is created, it makes use of this location configuration file.

In [14]:
dag = EODataAccessGateway(locations_conf_path=locations_filepath)

2025-10-20 11:53:42,905 eodag.config                     [INFO    ] Loading user configuration from: /home/sylvain/.config/eodag/eodag.yml
2025-10-20 11:53:42,962 eodag.core                       [INFO    ] Locations configuration loaded from /home/sylvain/workspace/eodag/docs/notebooks/tutos/eodag_workspace_locations_tiles/custom_locations.yml


We want to look for *Sentinel 2 Level-1C* products. We can check whether this collection is offered by *PEPS* (as configured in `eodag`). If so, *PEPS* is set as the provider used to search for products.

In [15]:
"S2_MSI_L1C" in dag.providers["peps"].collections

True

### Search

#### A single tile

Our target tile is `31TFK` and is located in the South-East of France. Its feature is retrieved from the shapefil to be displayed later on an interactive map.

In [16]:
targeted_tile_name = "31TFK"

# Get the targeted tile feature
targeted_tile = [
    sr
    for sr in shaperecs
    if sr.record["tile_id"] == "31TFK"
][0]

We search for all the products that intersect with the centroid of this tile.

In [17]:
products = dag.search_all(
    provider="peps",
    locations=dict(s2_tile_centroid="31TFK"),
    **default_search_criteria
)
print(f"{len(products)} were found given the above search criteria")

2025-10-20 11:54:27,610 eodag.core                       [INFO    ] Searching for all the products with provider peps and a maximum of 500 items per page.
2025-10-20 11:54:27,610 eodag.core                       [INFO    ] Iterate search over multiple pages: page #1
2025-10-20 11:54:27,611 eodag.core                       [INFO    ] Searching on provider peps
2025-10-20 11:54:27,620 eodag.search.qssearch            [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=POINT (4.9533 44.6422)&productType=S2MSI1C&maxRecords=500&page=1
2025-10-20 11:54:32,628 eodag.core                       [INFO    ] Found 6 result(s) on provider 'peps'


6 were found given the above search criteria


The products found are displayed on an interactive map along with the centroid of the targeted tile. A time player allows to see when the products were sensed.

In [18]:
# The GeoJSON representation has to be slightly adapted for the time slider
adapted_prods = products.as_geojson_object()
for feature in adapted_prods["features"]:
    feature["properties"]["time"] = feature["properties"]["start_datetime"]

# Create a map zoomed over the search area
fmap = folium.Map([44.5, 5], zoom_start=8)
# Add a layer that map the tile's centroid
folium.GeoJson(
    data=targeted_tile,
    tooltip = targeted_tile_name,
).add_to(fmap)
# Add layer that temporally maps the products found
TimestampedGeoJson(
    adapted_prods,
    transition_time=50,  # Transition duration in ms
    period="PT3H",  # Array of times, here every 3 hours
    duration="PT12H",  # Feature display duragion, here 6 hours
    time_slider_drag_update=True,  # Update the map when the slider is dragged
    auto_play=False,  # Don't auto play the animation
).add_to(fmap)
fmap

<div class="alert alert-info">

Note

Instead of using the tile's centroid we could have directly used its extent and filter the returned products to keep only those fully contained within the tile. Check out the section dedicated to [filtering products](../api_user_guide/6_crunch.ipynb) in the API user guide.

</div>

 

#### Multiple tiles

We can search for products that overlap with several tiles using a **regular expression**. We use the expression `"31T[CDE][MLK]"` to look for products over 9 different tiles (*31TCM*, *31TCL*, *31TCK*, *31TDM*, etc.) over France.

In [19]:
products = dag.search_all(
    provider="peps",
    locations=dict(s2_tile_centroid="31T[CDE][MLK]"),
    **default_search_criteria
)

2025-10-20 11:55:16,983 eodag.core                       [INFO    ] Searching for all the products with provider peps and a maximum of 500 items per page.
2025-10-20 11:55:16,984 eodag.core                       [INFO    ] Iterate search over multiple pages: page #1
2025-10-20 11:55:16,984 eodag.core                       [INFO    ] Searching on provider peps
2025-10-20 11:55:16,993 eodag.search.qssearch            [INFO    ] Sending search request: https://peps.cnes.fr/resto/api/collections/S2ST/search.json?startDate=2018-06-01&completionDate=2018-06-15&geometry=MULTIPOINT (3.7147 46.4567, 3.7032 45.5565, 3.6922 44.6568, 2.4122 46.4574, 2.4216 45.5572, 1.1109 46.4433, 1.1413 45.5436, 1.1703 44.6442, 2.4306 44.6575)&productType=S2MSI1C&maxRecords=500&page=1
2025-10-20 11:55:19,869 eodag.core                       [INFO    ] Found 32 result(s) on provider 'peps'


In [20]:
print(f"{len(products)} were found given the above search criteria")

32 were found given the above search criteria


The products are displayed on an interactive map. By hovering over them you can observe that the MGRS number of the tiles match with the regular expressions we used.

In [21]:
# Create a map zoomed over the search area
fmap = folium.Map([44.5, 1.5], zoom_start=6)
# Create a layer that maps the products found
folium.GeoJson(
    data=products,
    tooltip=folium.GeoJsonTooltip(
        fields=[
            "title",  # The product's title
            "grid:code",   # The tile number on the MGRS grid
        ]
    ),
).add_to(fmap)
fmap

This example has demonstrated the possibilities offered by `eodag` to easily select products from a tile grid by using regular expressions over their identifier.